In [1]:
# Install composer
# To install from source instead of the last release, use the below command instead:
# !pip install 'mosaicml[nlp]'

# Introduction

This tutorial will demonstrate how to fine-tune a pretrained HuggingFace transformer using the composer library! Composer provides a highly optimized and functional training loop and the ability to compose several methods that can accelerate training.

We will focus on fine-tuning a pretrained BERT-base model on the Stanford Sentiment Treebank v2 (SST-2) dataset. After fine-tuning, the BERT model should be able to determine if a setence has positive or negative sentiment.

Let's do this 🚀

In [2]:
import transformers
from torchmetrics import Accuracy
from torchmetrics.collections import MetricCollection
from composer.models.huggingface import HuggingFaceModel
from composer.metrics.nlp import LanguageCrossEntropy

# Define a Composer Model
# Create a BERT sequence classification model using HuggingFace transformsers
model = transformers.AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2) # in BERT hparams
tokenizer = transformers.AutoTokenizer.from_pretrained('bert-base-uncased') # from transformer_shared

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# Creating dataloaders

Next, we will download and tokenize the SST-2 datasets. 

In [3]:
import datasets
from multiprocessing import cpu_count

# Create BERT tokenizer
def tokenize_function(sample):
    return tokenizer(
        text=sample['sentence'],
        padding="max_length",
        max_length=256,
        truncation=True
    )

# Tokenize SST-2
sst2_dataset = datasets.load_dataset("glue", "sst2")
tokenized_sst2_dataset = sst2_dataset.map(tokenize_function,
                                          batched=True, 
                                          num_proc=cpu_count(),
                                          batch_size=100,
                                          remove_columns=['idx', 'sentence'])

# Split dataset into train and validation sets
train_dataset = tokenized_sst2_dataset["train"]
eval_dataset = tokenized_sst2_dataset["validation"]

Reusing dataset glue (/Users/austin/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /Users/austin/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-144ca3a878ff316e.arrow
Loading cached processed dataset at /Users/austin/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-6ad19558a8452aea.arrow
Loading cached processed dataset at /Users/austin/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-d458b1cf8bdc7cba.arrow
Loading cached processed dataset at /Users/austin/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-8391a9b34d4b57a9.arrow
Loading cached processed dataset at /Users/austin/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-f11afb9b427eb38c.arrow
Loading cached processed dataset at /Users/austin/.cache/hug

Here, we will create a PyTorch `DataLoader` for each of the datasets generated in the previous block.

In [4]:
from torch.utils.data import DataLoader
data_collator = transformers.data.data_collator.default_data_collator
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=False, drop_last=False, collate_fn=data_collator)
eval_dataloader = DataLoader(eval_dataset,batch_size=16, shuffle=False, drop_last=False, collate_fn=data_collator)

# define metrics and convert HF pretrained model into ComposerModel

In [5]:
metrics = [LanguageCrossEntropy(vocab_size=tokenizer.vocab_size), Accuracy()]
# Package as a composer model
composer_model = HuggingFaceModel(model, metrics=metrics)

# Optimizers and Learning Rate Schedulers

The last setup step is to create an optimizer and a learning rate scheduler. We will use PyTorch's AdamW optimizer and linear learning rate scheduler since these are typically used to fine-tune BERT on tasks such as SST-2.

In [6]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import LinearLR

optimizer = AdamW(
    params=composer_model.parameters(),
    lr=3e-5, betas=(0.9, 0.98),
    eps=1e-6, weight_decay=3e-6
)
linear_lr_decay = LinearLR(
    optimizer, start_factor=1.0,
    end_factor=0, total_iters=150
)

# Composer Trainer

We will now specify a Composer `Trainer` object and run our training! `Trainer` has many arguments that are described in our [documentation](https://docs.mosaicml.com/en/stable/api_reference/composer.trainer.trainer.html), but let's discuss the less obvious arguments used below:
- `max_duration` - a string specifying how long to train, either in terms of batches (e.g. '10ba' is 10 batches) or epochs (e.g. '1ep' is 1 epoch).
- `schedulers` - a list of PyTorch learning rate schedulers that will be composed together.
- `device` - specifies if the training will be done on CPU or GPU by using 'cpu' or 'gpu', respectively.
- `train_subset_num_batches` - specifies the number of training batches to use for each epoch. This is not a necessary argument but is useful for quickly testing code.
- `precision` - whether to do the training in full precision 'fp32' or mixed precision 'amp'. Mixed precision provides an almost 2x speedup in training time on certain hardware. If you get a P100, try `precision='amp'`!
- `seed` - sets the random seed for the training run, so the results are reproducible!

In [ ]:
import torch
from composer import Trainer

# Create Trainer Object
trainer = Trainer(
    model=composer_model, 
    train_dataloader=train_dataloader,
    eval_dataloader=eval_dataloader,
    max_duration="1ep",
    optimizers=optimizer,
    schedulers=[linear_lr_decay],
    device='gpu' if torch.cuda.is_available() else 'cpu',
    train_subset_num_batches=150,
    precision='fp32',
    seed=17
)
# Start training
trainer.fit()

/Users/austin/code/composer/composer/trainer/trainer.py:1247: UserWarning: NoSchedulerWarning: No schedulers were specified. The learning rate will be constant.
  warnings.warn(f"NoSchedulerWarning: No schedulers were specified. The learning rate will be constant.")


Epoch     0 train   0%|                         | 0/150 [00:00<?, ?it/s]                                      …

# Visualizing Results

Our model reaches almost 86% accuracy with only 100 iterations of training! Let's visualize a few samples from the validation set to see how our model performs.

In [ ]:
eval_batch = next(iter(eval_dataloader))

# Move batch to gpu
eval_batch = {k: v.cuda() if torch.cuda.is_available() else v for k, v in eval_batch.items()}
with torch.no_grad():
    predictions = composer_model(eval_batch)["logits"].argmax(dim=1)

# Visualize only 5 samples
predictions = predictions[:6]

label = ['negative', 'positive']
for i, prediction in enumerate(predictions[:6]):
    sentence = sst2_dataset["validation"][i]["sentence"]
    correct_label = label[sst2_dataset["validation"][i]["label"]]
    prediction_label = label[prediction]
    print(f"Sample: {sentence}")
    print(f"Label: {correct_label}")
    print(f"Prediction: {prediction_label}")
    print()

# Conclusion

This tutorial showed how to use the Composer `Trainer` to fine-tune a pre-trained BERT on a subset of the SST-2 dataset. We focused on the Composer's basic functionality, but there are many more tools such as easy to use gradient accumulation and multi-GPU training! Check out many other features at our [documentation](https://docs.mosaicml.com/en/latest).